In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import glove

In [74]:
wakati_file = 'toji.wakati'
with open("./데이터//"+wakati_file, 'r', encoding='UTF-8') as fp:
    corpus = fp.readlines()

공기행렬(Cooccurrence Matrix) 생성코드

In [75]:

vectorizer = CountVectorizer(min_df = 10, ngram_range=(1,1))
X = vectorizer.fit_transform(corpus)
Xc = X.T * X
Xc.setdiag(0)
result = Xc.toarray()

In [76]:
vectorizer.fit_transform(corpus)

<3439x661 sparse matrix of type '<class 'numpy.int64'>'
	with 21322 stored elements in Compressed Sparse Row format>

In [77]:
vectorizer.fit_transform(corpus).T * vectorizer.fit_transform(corpus)

<661x661 sparse matrix of type '<class 'numpy.int64'>'
	with 156359 stored elements in Compressed Sparse Column format>

In [78]:
dic = {}
for idx1, word1 in enumerate(result):
    tmpdic = {}
    for idx2, word2 in enumerate(word1):
        if word2 > 0:
            tmpdic[idx2] = word2
    dic[idx1] = tmpdic

"""
dic = { 0: { 0: 1.0,
             2: 3.5 },
        1: { 2: 0.5 },
        2: { 0: 3.5,
             1: 0.5,
             2: 1.2 }
      }
"""

In [85]:
import operator
vocab = sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))
vocab = [word[0] for word in vocab]

In [86]:
print(len(vocab))
print(vocab[:6])

661
['가까이', '가깝다', '가끔', '가다', '가락지', '가문']


In [87]:
import glove
model = glove.Glove(dic, d=100, alpha=0.75, x_max=100.0)
for epoch in range(25):
    err = model.train(batch_size=200, workers=4)
    if not epoch % 5:
        print("epoch %d, error %.3f" % (epoch, err), flush=True)

epoch 0, error 0.026
epoch 5, error 0.013
epoch 10, error 0.011
epoch 15, error 0.010
epoch 20, error 0.009


In [88]:
def most_similar(word, vocab, vecs, topn=10):
    query = vecs[vocab.index(word)]
    result = []
    for idx, vec in enumerate(vecs):
        if idx is not vocab.index(word):
            result.append((vocab[idx],1-cosine(query,vec)))
    result = sorted(result,key=lambda x: x[1],reverse=True)
    return result[:topn]

In [89]:
word = '가까이'
most_similar(word=word, vocab=vocab, vecs=model.W, topn=5)

[('흔들다', 0.9540346811468748),
 ('울리다', 0.953407736507298),
 ('마루', 0.9472240633891925),
 ('들리다', 0.9362093059635799),
 ('구석', 0.9337492783294976)]